In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Paths
DATASET_PATH = 'Dataset'  
TEST_DATASET_PATH = 'TestDataset' 

In [ ]:
# Parameters
IMAGE_SIZE = (224, 224)  # Input size for ResNet
BATCH_SIZE = 32
EPOCHS = 20

In [ ]:
# Load the ResNet50 model pre-trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Freeze the base model layers
base_model.trainable = False

In [ ]:
# Add custom layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification output layer


In [ ]:
# Create the model
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Preprocessing for training and validation
datagen = ImageDataGenerator(
    validation_split=0.1  # 10% for validation
)

In [ ]:
# Training set
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Binary classification
    subset='training'
)

# Validation set
validation_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Binary classification
    subset='validation'
)

# Test set
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_generator = test_datagen.flow_from_directory(
    TEST_DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Binary classification
    shuffle=False
)

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

In [ ]:
# Unfreeze some base model layers for fine-tuning
base_model.trainable = True
fine_tune_at = 100  # Unfreeze layers from this point

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile the model for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Fine-tune the model
history_fine = model.fit(
    train_generator,
    epochs=10,  # Adjust epochs for fine-tuning
    validation_data=validation_generator
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# Make predictions on the test set
predictions = model.predict(test_generator)

# Convert predictions to binary labels
predicted_classes = (predictions > 0.5).astype("int32")
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Display some results
print("\nSample Predictions:")
for i in range(10):  # Display results for the first 10 test samples
    print(f"True: {true_classes[i]}, Predicted: {predicted_classes[i][0]}")

In [ ]:
# Classification report and confusion matrix
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Classification report
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

# Confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()